# TP1 - Problema 2

### Grupo 11

Nelson Almeida a95652
<br>
Nuno Costa a97610

#### Problema 1

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
1. Cada reunião ocupa uma sala (enumeradas $ 1...S \,$) durante um “slot”  $1..T\,$ $(\text{hora},\text{dia})$.  
2.  Cada reunião tem associado um projeto (enumerados $1..P$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $1..C$.
3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $50\%$ do total de colaboradores do projeto. 
    
São “inputs” do problema:
1. Os parâmetros $S\,,\,T\,,\,P\,,\,C$
  
2. O conjunto de colaboradores de cada projeto, o seu líder e o número mínimo  de reuniões semanais.
    
3. A disponibilidade de cada participante, incluindo o lider. Essa disponibilidade   é um conjunto de “slots”  representada numa matriz booleana de acessibilidade com uma linha por cada participante $1..C$ e uma coluna por “slot” $\,1..T\,$

São critérios de optimização:
1. Maximizar o número de reuniões efetivamente realizadas
    
2. Minimizar o número médio de reuniões por participante.

In [123]:
from ortools.linear_solver import pywraplp
horario = pywraplp.Solver.CreateSolver("SCIP")
S, T, C, P = 3, (8,5), 3, 2
#3 salas, 8 slots p/dia 5 dias p/semana , 10 colaboradores, 5 projetos
colabPerProject = {0:[0,1,2],1:[]}
projectLeaders = {0:1}  
meetings = {0:10}

maxMeets = horario.IntVar(0,sum(meetings[x] for x in meetings), "maxMeetings") #variavel para maximizar o numero de reuniões por semana

#inicialização da matriz
x = {}
for s in range(S):
    x[s]={}
    for d in range(T[1]):
        x[s][d]={}
        for h in range(T[0]):
            x[s][d][h]={}
            for p in range(P):
                x[s][d][h][p] = horario.BoolVar(f"x[{s}][{d}][{h}][{p}]")
        
y = {}
for c in range(C):
    y[c]={}
    for d in range(T[1]):
        y[c][d]={}
        for h in range(T[0]):
            y[c][d][h]={}
            for p in range(P):
                y[c][d][h][p]=horario.BoolVar(f"y[{c}][{d}][{h}][{p}]")

Passamos agora à modelação das restrições e à sua introdução no *solver*.

A restrição:

1. O quorum por reunião é igual ou superior a 50%

$$
\forall_{d < D} \;\forall_{p < P} \;\forall_{h < H} \;\sum_{c < C} y_{c,d,h,p} \geq 0.5 \cdot len(colabsPerProject[p])
$$


In [124]:
for d in range(T[1]):
    for p in range(P):
        for h in range(T[0]):
            horario.Add(sum(y[c][d][h][p] for c in range(C))>=0.5*len(colabPerProject[p]))       

A restrição:

2. Verificar que o número mínimo de reuniões semanais é satisfeito.

$$
\forall_{p < P} \sum_{s < S, d < D, h < H} x_{s,d,h,p} \geq meetings[p]
$$

In [125]:
#verificar que o número mínimo de reuniões semanais é satisfeito
for projeto in range(P):
    horario.Add(sum([x[s][d][h][projeto] for s in range(S) for d in range(T[1]) for h in range(T[0])])>=meetings[projeto])

A restrição:

3. Verificar que o líder participa em todas as reuniões

$$
\forall_ {d<D} \;\forall_{p < P} \;\forall_{h < H} 
$$
 
ACABAR

In [126]:
#verificar que o lider participa em todas as reuniões
for d in range(T[1]):
    for p in range(P):
        for h in range(T[0]):
            lider = projectLeaders[p]
            horario.Add(sum([x[s][d][h][p]])==y[lider][d][h][p])

A restrição:

4. Cada sala só é utilizada para uma reunião de cada vez

$$
\forall_{s < S} \;\forall_{d < D} \;\forall_{h < H} \sum_{p < P} x_{s,d,h,p} \leq 1
$$

TESTAR

In [1]:
for s in range(S):
    for d in range(T[1]):
        for h in range(T[0]):
            horario.Add(sum([x[s][d][h][p] for p in range(P)])<=1)

NameError: name 'S' is not defined

A restrição:

5. Verificar que o só existe um projeto alocado a cada colaborador de cada vez

$$
\forall_{c < C} \;\forall_{d < D} \;\forall_{h < H} \sum_{p < P} y_{c,d,h,p} \leq 1
$$

In [128]:
for c in range(C):
    for d in range(T[1]):
        for h in range(T[0]):
            horario.Add(sum([y[c][d][h][p] for p in range(P)])<=1)

A restrição:

6. Verificar que os projetos alocados a cada colaborador são os originalmente selecionados

$$
\forall_{c < C} \;\forall_{d < D} \;\forall_{h < H} \;\forall_{p < P} 
$$

ACABAR

In [129]:
for c in range(C):
    for d in range(T[1]):
        for h in range(T[0]):
            for p in range(P):
                if c not in colabPerProject[p]:
                    horario.Add(y[c][d][h][p]==0)
                    

*Maximizar* o número de reuniões realizadas por projeto


In [130]:
horario.Maximize(sum(x[s][d][h][p] for s in range(S) for d in range(T[1]) for h in range(T[0]) for p in range(P)))

Minimizar o número médio de reuniões semanais por colaborador

In [131]:
horario.Minimize(sum(y[c][d][h][p] for c in range(C) for d in range(T[1]) for h in range(T[0]) for p in range(P))/C)

Solução do Problema

In [135]:
from tabulate import tabulate

status = horario.Solve()
if status== pywraplp.Solver.OPTIMAL:
    print("Solução encontrada")
                    
    tabela = [[] for hora in range(T[0]+1)]  #horas+1 pois queremos a primeira row para colocar os dias
    #definir os dias
    tabela[0].append("")
    semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
    for d in range(T[1]):
        tabela[0].append(semana[d])
    
    #definir as horas de trabalho
    for h in range(1,T[0]+1):
        tabela[h].append(f"{int(h)+8}:00h")
        
    #começar a preencher a tabela
    #sala, projeto, colaboradores presentes
    for d in range(T[1]): #para cada dia da semana
        for h in range(T[0]):
            for p in range(P):
                working,salaTemp =0,0
                for s in range(S):
                    if round(x[s][d][h][p].solution_value())==1:
                        working =1
                        salaTemp=s
                if working==1:
                    tabela[h+1].append(f"Projeto:{p}\nSala:{salaTemp}\nColaboradores:{1}")
    
    
    print(tabulate(tabela, tablefmt="fancy_grid"))
    
    
    
else:
    print("Não foi encontrada solução")

Solução encontrada
╒════════╤═════════════════╤═════════════════╤═════════════════╤═════════════════╤═════════════════╕
│        │ Segunda         │ Terça           │ Quarta          │ Quinta          │ Sexta           │
├────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│ 9:00h  │ Projeto:0       │ Projeto:0       │ Projeto:0       │ Projeto:0       │ Projeto:0       │
│        │ Sala:1          │ Sala:1          │ Sala:1          │ Sala:1          │ Sala:1          │
│        │ Colaboradores:1 │ Colaboradores:1 │ Colaboradores:1 │ Colaboradores:1 │ Colaboradores:1 │
├────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│ 10:00h │ Projeto:0       │ Projeto:0       │ Projeto:0       │ Projeto:0       │ Projeto:0       │
│        │ Sala:1          │ Sala:1          │ Sala:1          │ Sala:1          │ Sala:1          │
│        │ Colaboradores:1 │ Colaboradores:1 │ Colaboradores:1 │ Colabor